In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('/content/Flipit_news_colab2/flipitnews-data.csv')

In [7]:
df.head()

,Category,Article
0,Technology,tv future in the hands of viewers with home th...
1,Business,worldcom boss left books alone former worldc...
2,Sports,tigers wary of farrell gamble leicester say ...
3,Sports,yeading face newcastle in fa cup premiership s...
4,Entertainment,ocean s twelve raids box office ocean s twelve...


In [8]:
df.shape

(2225, 2)

In [9]:
words_num = [len(df['Article'][x].split()) for x in range(len(df['Article']))]

In [10]:
df_words = pd.DataFrame(words_num, columns=['words_num'])

In [11]:
df_words.describe()

,words_num
count,2225.000000
mean,390.295281
std,241.753128
min,90.000000
25%,250.000000
50%,337.000000
75%,479.000000
max,4492.000000


In [12]:
len(df['Article'][5].split())

633

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


In [13]:
train_text, test_text, train_labels, test_labels = train_test_split(df['Article'], df['Category'], test_size=0.2, random_state=42)

In [14]:
len(train_text.to_list())

1780

In [15]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# tokenize a set of texts
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
len(train_encodings['input_ids'][2])

512

In [18]:
# convert the lists to tensors
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#train_labels = train_labels.astype('category').cat.codes
#test_labels = test_labels.astype('category').cat.codes
train_dataset = NewsDataset(train_encodings, train_labels.to_list())
test_dataset = NewsDataset(test_encodings, test_labels.to_list())

In [19]:
train_dataset[0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [20]:
len(set(train_labels))

5

In [21]:
# Load the BERT model
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(train_labels)))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# define the optimizer and trainer
from transformers import AdamW
from torch.utils.data import DataLoader

# create the data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# fine tune the model
from transformers import get_scheduler
from tqdm import tqdm

# define the learning rate scheduler
num_epochs = 2
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())



Epoch 1: 100%|██████████| 112/112 [02:47<00:00,  1.50s/it, loss=0.0182]


In [24]:
# evaluate the model
from sklearn.metrics import accuracy_score

model.eval()
predictions = []
real_values = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        predictions.extend(predicted.cpu().numpy())
        real_values.extend(labels.cpu().numpy())

# calculate the accuracy
accuracy = accuracy_score(real_values, predictions)
print(f'Test Accuracy: {accuracy}')


Test Accuracy: 0.9842696629213483


In [25]:
# save the model
model.save_pretrained('news_model')
tokenizer.save_pretrained('news_model')

('news_model/tokenizer_config.json',
 'news_model/special_tokens_map.json',
 'news_model/vocab.txt',
 'news_model/added_tokens.json')

In [26]:
# load the model for inference
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('news_model')
tokenizer = BertTokenizer.from_pretrained('news_model')

# prepare the input
text = 'The stock market is going up'
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
outputs = model(**inputs)
_, predicted = torch.max(outputs.logits, 1)
print(f'Predicted category: {predicted.item()}')

Predicted category: 0
